# Problem 3 Naive Bayes

## 3.1 implement a Naive Bayes classifier

In [1]:
import sys
import math
import itertools
from collections import Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time
from copy import deepcopy

In [2]:
class Model:
    def __init__(self, wordlist):
        self.wordlist = wordlist

    def count_labels(self, data):
        """
        Count the number of positive labels and negative labels.
        Returns (a tuple or a numpy array of two elements):
            * negative_count: a non-negative integer, which represents the number of negative labels (non-spam emails);
            * positive_count: a non-negative integer, which represents the number of positive labels (spam emails).
        """
        # TODO
        result = np.zeros((2))
        pos = 0
        neg = 0
        for i in range(len(data)):
            if data[i][0] == 1:
                pos += 1
            else:
                neg += 1
                
        result[0] = neg
        result[1] = pos
        print("number of spam email ",result[1])
        return result
    
    def count_words(self, wordlist, data):
        """
        Count the number of times that each word appears in emails under a given label.
        Returns (a numpy array):
            * word_counts: a numpy array with shape (2, L), where L is the length of $wordlist,
                - word_counts[0, i] represents the number of times that word $wordlist[i] appears in non-spam (negative) emails, and
                - word_counts[1, i] represents the number of times that word $wordlist[i] appears in spam (positive) emails.
        """
        # TODO
        L = len(wordlist)
        word_counts = np.zeros((2,L))
#         for i in range(L):
#             for j in data:
#                 if j[0] == 0:
#                     #non spam
#                     for word in j[1]:
#                         if word == wordlist[i]:
#                             word_counts[0][i] += 1
#                             break
#                 else:
#                     #spam
#                     for word in j[1]:
#                         if word == wordlist[i]:
#                             word_counts[1][i] += 1
#                             break
        #transform data into ndarray
        data = np.asarray(data)
        print("shape of data: ",data.shape)
        spam_data = data[np.where(data[:,0]==1)][:,1]
        nons_data = data[np.where(data[:,0]==0)][:,1]
        
        
#         print("shape of spam data: ",len(spam_data))
        
        for j in range(L):
            for i in range(len(spam_data)):
#                 row = np.array(spam_data[i])
#                 values,counts = np.unique(row,return_counts=True)
#                 row_dict = dict(zip(values,counts))
                row_set = set(spam_data[i])
                if wordlist[j] in row_set:
                    word_counts[1][j]+=1
        
        for j in range(L):
            for i in range(len(nons_data)):
#                 row = np.array(nons_data[i])
#                 values,counts = np.unique(row,return_counts=True)
#                 row_dict = dict(zip(values,counts))
                row_set = set(nons_data[i])
                if wordlist[j] in row_set:
                    word_counts[0][j]+=1
        
        return word_counts
                    
#         values, counts = np.unique(spam_data, return_counts=True)
#         spam_dict = dict(zip(values,counts))
        
#         values, counts = np.unique(nons_data, return_counts=True)
#         nons_dict = dict(zip(values,counts))
        
#         for i in range(L):
#             word = wordlist[i]
#             if word in spam_dict:
#                 word_counts[1,i] = spam_dict[word]
#                 print(word,spam_dict[word])
#             else:
#                 word_counts[1,i] = 0
            
#             if word in nons_dict:
#                 word_counts[0,i] = nons_dict[word]
#                 print(word,nons_dict[word])
#             else:
#                 word_counts[0,i] = 0
        
# #         print("word_counts: ",word_counts)
#         return word_counts

    def calculate_probability(self, label_counts, word_counts):
        """
        Calculate the probabilities, both the prior and likelihood.
        Returns (a pair of numpy array):
            * prior_probs: a numpy array with shape (2, ), only two elements, where
                - prior_probs[0] is the prior probability of negative labels, and
                - prior_probs[1] is the prior probability of positive labels.
            * likelihood_probs: a numpy array with shape (2, L), where L is the length of the word list,
                - likelihood_probs[0, i] represents the likelihood probability of the $i-th word in the word list, given that the email is non-spam (negative), and
                - likelihood_probs[1, i] represents the likelihood probability of the $i-th word in the word list, given that the email is spam (positive).
        """
        # TODO
        L = word_counts.shape[1]
        prior_probs = np.zeros((2))
        likelihood_probs = np.zeros((2,L))
        prior_probs[0] = label_counts[0] / (label_counts[0]+ label_counts[1])
        prior_probs[1] = label_counts[1] / (label_counts[0]+ label_counts[1])                       
        
        for i in range(word_counts.shape[1]):
            likelihood_probs[0,i] = (word_counts[0,i]+1)/(label_counts[0]+2)
            
            likelihood_probs[1,i] = (word_counts[1,i]+1)/(label_counts[1]+2)
#             print("index ",i," this word count in non-spam + 1",word_counts[0,i]+1,"non-spam total: ",label_counts[0]+2)
#             print("index ",i,"this word count in spam + 1",word_counts[1,i]+1,"spam total: ",label_counts[1]+2)
#             print("index ",i,"likelihood_probs for index i",likelihood_probs[0,i],likelihood_probs[1,i])
            
        return prior_probs,likelihood_probs
        
    def fit(self, data):
        label_counts = self.count_labels(data)
        word_counts = self.count_words(self.wordlist, data)
        
        self.label_counts = label_counts
        self.word_counts = word_counts
        self.prior_probs, self.likelihood_probs = self.calculate_probability(label_counts, word_counts)

        # TO AVOID NUMBER OVERFLOW here we use log probability instead.
        self.log_prior_probs = np.log(self.prior_probs)
        self.log_likelihood_probs = np.log(self.likelihood_probs)
#         np.dstack([np.log(1 - self.likelihood_probs), np.log(self.likelihood_probs)])
        
    def predict(self, x):
        """
        Predict whether email $x is a spam or not.
        Returns:
            * y: a boolean value indicating whether $x is a spam or not.
        """        
        p_x_spam = 0
        p_x_non = 0
        
        for word in self.wordlist:
            index = self.wordlist.index(word)
            if word in x:
                p_x_spam += self.log_likelihood_probs[1][index]
                p_x_non += self.log_likelihood_probs[0][index]
            else:
                try:
                    p_x_spam += math.log((1-self.likelihood_probs[1][index]))
                    p_x_non += math.log((1-self.likelihood_probs[0][index]))
                except:
                    print("math error, the prob is ",self.likelihood_probs[1][index], "and",(self.likelihood_probs[0][index]))
#                     print("p_x_spam is ",)
                    break
        
        p_x_spam += self.log_prior_probs[1]
        p_x_non += self.log_prior_probs[0]
        if p_x_spam >= p_x_non:
            return True
        else:
            return False

In [3]:
def read_data(filename):
    """
    Read the dataset from the file given by name $filename.
    The returned object should be a list of pairs of data. In each pair: the first element is 1 (for spam emails) 
    or 0 (for non-spam emails), the second element is a list of words in the email.
    The returned list: 
        [
            (1 , ['a', 'b', 'c']),
            (0, ['d', 'e', 'f']),
            ...
        ]
    """
    
    result = []
    
    df = pd.read_csv(filename,sep='\n',header=None,names=["email"])
    df = pd.DataFrame(df.email.str.split(' ',1).tolist(),
                                 columns = ['label','text']) 
    for row in df.itertuples(index=False):
        label = int(row.label)
        text = row.text.split(' ')
        new_data = (label,text)
        result.append(new_data)
        
    return result

    
    
def split_train(original_train_data, size=4000):
    return original_train_data[:size], original_train_data[size:]


def create_wordlist(original_train_data, threshold=26):
    """
    Create a word list from the original training set.
    Only get a word if it appears in at least $threshold emails.
    Returns:
        * a python list containing all the words that occur in at least $threshold emails.
    """
    dic = {}
    
    for i in range(len(original_train_data)):
        texts = original_train_data[i][1]
#         print(texts)
        texts = set(texts)
        for word in texts:
            if word in dic:
                dic[word]+=1
            else:
                dic[word] =1
        
    wordlist = []
    for i in dic.keys():
        if dic[i] >= threshold:
            wordlist.append(i)
    
    return wordlist

In [4]:
# threshold to determine whether to include a word in the dictionary/wordlist.
# ie. only words with frequency higher than threshold are included
THRESHOLD = 26

In [5]:
original_train_data = read_data('spam_train.txt')


# further split the data into a training set and a validation set
train_data, val_data = split_train(original_train_data)
print("test")
# Create the word list.
wordlist = create_wordlist(original_train_data, 26)
print("Total # of words:", len(wordlist))

# fit the model using train_data
model = Model(wordlist)
model.fit(train_data)
# print("shape of log: ",model.log_likelihood_probs.shape)

# TODO
# calculate the error rate on val_data (when threshold=26)
# print out the error rate
result = {"spam":0,"non-spam":0}
error_count =0
for i in range(len(val_data)):
    is_spam = model.predict(val_data[i][1])
    if is_spam == True:
        if val_data[i][0] == 0:
            error_count  += 1
    else:
        if val_data[i][0] == 1:
            error_count  += 1

error_percentage = (error_count / len(val_data))*100
print("Validation error, # = {:>4d}, % = {:>8.4f}%.".format(error_count, error_percentage))

test
Total # of words: 3048
number of spam email  1255.0
shape of data:  (4000, 2)
index  0  this word count in non-spam + 1 227.0 non-spam total:  2747.0
index  0 this word count in spam + 1 332.0 spam total:  1257.0
index  0 likelihood_probs for index i 0.08263560247542774 0.26412092283214
index  1  this word count in non-spam + 1 92.0 non-spam total:  2747.0
index  1 this word count in spam + 1 80.0 spam total:  1257.0
index  1 likelihood_probs for index i 0.03349108117946851 0.06364359586316627
index  2  this word count in non-spam + 1 23.0 non-spam total:  2747.0
index  2 this word count in spam + 1 55.0 spam total:  1257.0
index  2 likelihood_probs for index i 0.008372770294867127 0.04375497215592681
index  3  this word count in non-spam + 1 1114.0 non-spam total:  2747.0
index  3 this word count in spam + 1 652.0 spam total:  1257.0
index  3 likelihood_probs for index i 0.4055333090644339 0.5186953062848051
index  4  this word count in non-spam + 1 30.0 non-spam total:  2747.0
i

index  232 this word count in spam + 1 137.0 spam total:  1257.0
index  232 likelihood_probs for index i 0.09501274117218785 0.10898965791567224
index  233  this word count in non-spam + 1 120.0 non-spam total:  2747.0
index  233 this word count in spam + 1 8.0 spam total:  1257.0
index  233 likelihood_probs for index i 0.04368401892974154 0.006364359586316627
index  234  this word count in non-spam + 1 546.0 non-spam total:  2747.0
index  234 this word count in spam + 1 121.0 spam total:  1257.0
index  234 likelihood_probs for index i 0.19876228613032398 0.09626093874303898
index  235  this word count in non-spam + 1 398.0 non-spam total:  2747.0
index  235 this word count in spam + 1 83.0 spam total:  1257.0
index  235 likelihood_probs for index i 0.14488532945030944 0.066030230708035
index  236  this word count in non-spam + 1 176.0 non-spam total:  2747.0
index  236 this word count in spam + 1 33.0 spam total:  1257.0
index  236 likelihood_probs for index i 0.06406989443028759 0.02

index  467 likelihood_probs for index i 0.1842009464870768 0.10739856801909307
index  468  this word count in non-spam + 1 22.0 non-spam total:  2747.0
index  468 this word count in spam + 1 9.0 spam total:  1257.0
index  468 likelihood_probs for index i 0.008008736803785948 0.007159904534606206
index  469  this word count in non-spam + 1 20.0 non-spam total:  2747.0
index  469 this word count in spam + 1 5.0 spam total:  1257.0
index  469 likelihood_probs for index i 0.0072806698216235895 0.003977724741447892
index  470  this word count in non-spam + 1 101.0 non-spam total:  2747.0
index  470 this word count in spam + 1 106.0 spam total:  1257.0
index  470 likelihood_probs for index i 0.03676738259919913 0.08432776451869531
index  471  this word count in non-spam + 1 204.0 non-spam total:  2747.0
index  471 this word count in spam + 1 111.0 spam total:  1257.0
index  471 likelihood_probs for index i 0.07426283218056061 0.0883054892601432
index  472  this word count in non-spam + 1 57.

index  722  this word count in non-spam + 1 74.0 non-spam total:  2747.0
index  722 this word count in spam + 1 56.0 spam total:  1257.0
index  722 likelihood_probs for index i 0.02693847834000728 0.04455051710421639
index  723  this word count in non-spam + 1 31.0 non-spam total:  2747.0
index  723 this word count in spam + 1 18.0 spam total:  1257.0
index  723 likelihood_probs for index i 0.011285038223516564 0.014319809069212411
index  724  this word count in non-spam + 1 15.0 non-spam total:  2747.0
index  724 this word count in spam + 1 30.0 spam total:  1257.0
index  724 likelihood_probs for index i 0.005460502366217692 0.02386634844868735
index  725  this word count in non-spam + 1 14.0 non-spam total:  2747.0
index  725 this word count in spam + 1 8.0 spam total:  1257.0
index  725 likelihood_probs for index i 0.005096468875136513 0.006364359586316627
index  726  this word count in non-spam + 1 14.0 non-spam total:  2747.0
index  726 this word count in spam + 1 12.0 spam total:

index  967 likelihood_probs for index i 0.016381507098653075 0.04455051710421639
index  968  this word count in non-spam + 1 103.0 non-spam total:  2747.0
index  968 this word count in spam + 1 62.0 spam total:  1257.0
index  968 likelihood_probs for index i 0.03749544958136149 0.049323786793953855
index  969  this word count in non-spam + 1 25.0 non-spam total:  2747.0
index  969 this word count in spam + 1 19.0 spam total:  1257.0
index  969 likelihood_probs for index i 0.009100837277029487 0.015115354017501989
index  970  this word count in non-spam + 1 22.0 non-spam total:  2747.0
index  970 this word count in spam + 1 18.0 spam total:  1257.0
index  970 likelihood_probs for index i 0.008008736803785948 0.014319809069212411
index  971  this word count in non-spam + 1 168.0 non-spam total:  2747.0
index  971 this word count in spam + 1 93.0 spam total:  1257.0
index  971 likelihood_probs for index i 0.06115762650163815 0.07398568019093078
index  972  this word count in non-spam + 1 

...

index  2056  this word count in non-spam + 1 31.0 non-spam total:  2747.0
index  2056 this word count in spam + 1 4.0 spam total:  1257.0
index  2056 likelihood_probs for index i 0.011285038223516564 0.0031821797931583136
index  2057  this word count in non-spam + 1 95.0 non-spam total:  2747.0
index  2057 this word count in spam + 1 9.0 spam total:  1257.0
index  2057 likelihood_probs for index i 0.03458318165271205 0.007159904534606206
index  2058  this word count in non-spam + 1 16.0 non-spam total:  2747.0
index  2058 this word count in spam + 1 20.0 spam total:  1257.0
index  2058 likelihood_probs for index i 0.005824535857298871 0.015910898965791568
index  2059  this word count in non-spam + 1 20.0 non-spam total:  2747.0
index  2059 this word count in spam + 1 6.0 spam total:  1257.0
index  2059 likelihood_probs for index i 0.0072806698216235895 0.00477326968973747
index  2060  this word count in non-spam + 1 51.0 non-spam total:  2747.0
index  2060 this word count in spam + 1 3

index  2320 this word count in spam + 1 22.0 spam total:  1257.0
index  2320 likelihood_probs for index i 0.013105205678922462 0.017501988862370723
index  2321  this word count in non-spam + 1 8.0 non-spam total:  2747.0
index  2321 this word count in spam + 1 15.0 spam total:  1257.0
index  2321 likelihood_probs for index i 0.0029122679286494356 0.011933174224343675
index  2322  this word count in non-spam + 1 16.0 non-spam total:  2747.0
index  2322 this word count in spam + 1 19.0 spam total:  1257.0
index  2322 likelihood_probs for index i 0.005824535857298871 0.015115354017501989
index  2323  this word count in non-spam + 1 8.0 non-spam total:  2747.0
index  2323 this word count in spam + 1 15.0 spam total:  1257.0
index  2323 likelihood_probs for index i 0.0029122679286494356 0.011933174224343675
index  2324  this word count in non-spam + 1 19.0 non-spam total:  2747.0
index  2324 this word count in spam + 1 7.0 spam total:  1257.0
index  2324 likelihood_probs for index i 0.00691

index  2598  this word count in non-spam + 1 23.0 non-spam total:  2747.0
index  2598 this word count in spam + 1 5.0 spam total:  1257.0
index  2598 likelihood_probs for index i 0.008372770294867127 0.003977724741447892
index  2599  this word count in non-spam + 1 37.0 non-spam total:  2747.0
index  2599 this word count in spam + 1 19.0 spam total:  1257.0
index  2599 likelihood_probs for index i 0.01346923917000364 0.015115354017501989
index  2600  this word count in non-spam + 1 37.0 non-spam total:  2747.0
index  2600 this word count in spam + 1 23.0 spam total:  1257.0
index  2600 likelihood_probs for index i 0.01346923917000364 0.018297533810660304
index  2601  this word count in non-spam + 1 20.0 non-spam total:  2747.0
index  2601 this word count in spam + 1 6.0 spam total:  1257.0
index  2601 likelihood_probs for index i 0.0072806698216235895 0.00477326968973747
index  2602  this word count in non-spam + 1 26.0 non-spam total:  2747.0
index  2602 this word count in spam + 1 6.

index  2879 this word count in spam + 1 2.0 spam total:  1257.0
index  2879 likelihood_probs for index i 0.010921004732435385 0.0015910898965791568
index  2880  this word count in non-spam + 1 21.0 non-spam total:  2747.0
index  2880 this word count in spam + 1 5.0 spam total:  1257.0
index  2880 likelihood_probs for index i 0.007644703312704768 0.003977724741447892
index  2881  this word count in non-spam + 1 23.0 non-spam total:  2747.0
index  2881 this word count in spam + 1 8.0 spam total:  1257.0
index  2881 likelihood_probs for index i 0.008372770294867127 0.006364359586316627
index  2882  this word count in non-spam + 1 14.0 non-spam total:  2747.0
index  2882 this word count in spam + 1 16.0 spam total:  1257.0
index  2882 likelihood_probs for index i 0.005096468875136513 0.012728719172633254
index  2883  this word count in non-spam + 1 29.0 non-spam total:  2747.0
index  2883 this word count in spam + 1 5.0 spam total:  1257.0
index  2883 likelihood_probs for index i 0.0105569

Validation error, # =   61, % =   6.1000%.


In [ ]:
1-math.log(381/1256)

## 3.2 try different thresholds, find the optimal threshold (which gives minimum validation error), print out the test error at the optimal threshold

In [6]:
def compute_error_rate(model, data):
    
    error_count = sum([y != model.predict(x) for y, x in data])
    return 100.0 * error_count / len(data)

In [7]:
thresholds = list(range(1, 35))
train_error = []
val_error = []
test_error = []
original_train_data = read_data('spam_train.txt')
train_data, val_data = split_train(original_train_data)
test_data = read_data('spam_test.txt')

In [ ]:
for th in thresholds[::-1]:
    print('With threshold {}....'.format(th))
    time1 = time.time()

    # vocabulary selection
    wordlist = create_wordlist(original_train_data, th)

    # fit model using the wordlist
    model = Model(wordlist)
    model.fit(train_data)
    
    print("finish training data ...")
    # compute classification error rates
    err_train = compute_error_rate(model, train_data)
    err_val = compute_error_rate(model, val_data)
    err_test = compute_error_rate(model, test_data)

    # store results for plotting
    train_error.append(err_train)
    val_error.append(err_val)
    test_error.append(err_test)

    time2 = time.time()
    print("train:{} val:{} test:{} len(V)={}".format(err_train, err_val, err_test, len(wordlist)))
    print('time: {}'.format(time2 - time1))

With threshold 34....
number of spam email  1255.0
shape of data:  (4000, 2)
index  0  this word count in non-spam + 1 227.0 non-spam total:  2747.0
index  0 this word count in spam + 1 332.0 spam total:  1257.0
index  0 likelihood_probs for index i 0.08263560247542774 0.26412092283214
index  1  this word count in non-spam + 1 92.0 non-spam total:  2747.0
index  1 this word count in spam + 1 80.0 spam total:  1257.0
index  1 likelihood_probs for index i 0.03349108117946851 0.06364359586316627
index  2  this word count in non-spam + 1 23.0 non-spam total:  2747.0
index  2 this word count in spam + 1 55.0 spam total:  1257.0
index  2 likelihood_probs for index i 0.008372770294867127 0.04375497215592681
index  3  this word count in non-spam + 1 1114.0 non-spam total:  2747.0
index  3 this word count in spam + 1 652.0 spam total:  1257.0
index  3 likelihood_probs for index i 0.4055333090644339 0.5186953062848051
index  4  this word count in non-spam + 1 30.0 non-spam total:  2747.0
index  

index  241 likelihood_probs for index i 0.12522752093192574 0.12649164677804295
index  242  this word count in non-spam + 1 253.0 non-spam total:  2747.0
index  242 this word count in spam + 1 120.0 spam total:  1257.0
index  242 likelihood_probs for index i 0.0921004732435384 0.0954653937947494
index  243  this word count in non-spam + 1 479.0 non-spam total:  2747.0
index  243 this word count in spam + 1 145.0 spam total:  1257.0
index  243 likelihood_probs for index i 0.17437204222788497 0.11535401750198886
index  244  this word count in non-spam + 1 92.0 non-spam total:  2747.0
index  244 this word count in spam + 1 37.0 spam total:  1257.0
index  244 likelihood_probs for index i 0.03349108117946851 0.0294351630867144
index  245  this word count in non-spam + 1 89.0 non-spam total:  2747.0
index  245 this word count in spam + 1 116.0 spam total:  1257.0
index  245 likelihood_probs for index i 0.032398980706224975 0.09228321400159109
index  246  this word count in non-spam + 1 52.0 

index  477  this word count in non-spam + 1 49.0 non-spam total:  2747.0
index  477 this word count in spam + 1 15.0 spam total:  1257.0
index  477 likelihood_probs for index i 0.017837641062977794 0.011933174224343675
index  478  this word count in non-spam + 1 249.0 non-spam total:  2747.0
index  478 this word count in spam + 1 100.0 spam total:  1257.0
index  478 likelihood_probs for index i 0.09064433927921368 0.07955449482895784
index  479  this word count in non-spam + 1 285.0 non-spam total:  2747.0
index  479 this word count in spam + 1 1.0 spam total:  1257.0
index  479 likelihood_probs for index i 0.10374954495813615 0.0007955449482895784
index  480  this word count in non-spam + 1 169.0 non-spam total:  2747.0
index  480 this word count in spam + 1 132.0 spam total:  1257.0
index  480 likelihood_probs for index i 0.06152165999271933 0.10501193317422435
index  481  this word count in non-spam + 1 75.0 non-spam total:  2747.0
index  481 this word count in spam + 1 59.0 spam to

index  741 this word count in spam + 1 85.0 spam total:  1257.0
index  741 likelihood_probs for index i 0.16963960684382964 0.06762132060461416
index  742  this word count in non-spam + 1 57.0 non-spam total:  2747.0
index  742 this word count in spam + 1 8.0 spam total:  1257.0
index  742 likelihood_probs for index i 0.02074990899162723 0.006364359586316627
index  743  this word count in non-spam + 1 129.0 non-spam total:  2747.0
index  743 this word count in spam + 1 56.0 spam total:  1257.0
index  743 likelihood_probs for index i 0.04696032034947215 0.04455051710421639
index  744  this word count in non-spam + 1 31.0 non-spam total:  2747.0
index  744 this word count in spam + 1 3.0 spam total:  1257.0
index  744 likelihood_probs for index i 0.011285038223516564 0.002386634844868735
index  745  this word count in non-spam + 1 109.0 non-spam total:  2747.0
index  745 this word count in spam + 1 22.0 spam total:  1257.0
index  745 likelihood_probs for index i 0.03967965052784856 0.017

index  1006 this word count in spam + 1 36.0 spam total:  1257.0
index  1006 likelihood_probs for index i 0.056789224608664 0.028639618138424822
index  1007  this word count in non-spam + 1 146.0 non-spam total:  2747.0
index  1007 this word count in spam + 1 57.0 spam total:  1257.0
index  1007 likelihood_probs for index i 0.053148889697852204 0.045346062052505964
index  1008  this word count in non-spam + 1 89.0 non-spam total:  2747.0
index  1008 this word count in spam + 1 16.0 spam total:  1257.0
index  1008 likelihood_probs for index i 0.032398980706224975 0.012728719172633254
index  1009  this word count in non-spam + 1 25.0 non-spam total:  2747.0
index  1009 this word count in spam + 1 3.0 spam total:  1257.0
index  1009 likelihood_probs for index i 0.009100837277029487 0.002386634844868735
index  1010  this word count in non-spam + 1 279.0 non-spam total:  2747.0
index  1010 this word count in spam + 1 96.0 spam total:  1257.0
index  1010 likelihood_probs for index i 0.101565

index  1271  this word count in non-spam + 1 42.0 non-spam total:  2747.0
index  1271 this word count in spam + 1 55.0 spam total:  1257.0
index  1271 likelihood_probs for index i 0.015289406625409537 0.04375497215592681
index  1272  this word count in non-spam + 1 136.0 non-spam total:  2747.0
index  1272 this word count in spam + 1 25.0 spam total:  1257.0
index  1272 likelihood_probs for index i 0.04950855478704041 0.01988862370723946
index  1273  this word count in non-spam + 1 121.0 non-spam total:  2747.0
index  1273 this word count in spam + 1 51.0 spam total:  1257.0
index  1273 likelihood_probs for index i 0.044048052420822714 0.0405727923627685
index  1274  this word count in non-spam + 1 91.0 non-spam total:  2747.0
index  1274 this word count in spam + 1 29.0 spam total:  1257.0
index  1274 likelihood_probs for index i 0.03312704768838733 0.023070803500397773
index  1275  this word count in non-spam + 1 13.0 non-spam total:  2747.0
index  1275 this word count in spam + 1 74

index  1535 likelihood_probs for index i 0.039315617036767385 0.17342879872712808
index  1536  this word count in non-spam + 1 57.0 non-spam total:  2747.0
index  1536 this word count in spam + 1 62.0 spam total:  1257.0
index  1536 likelihood_probs for index i 0.02074990899162723 0.049323786793953855
index  1537  this word count in non-spam + 1 16.0 non-spam total:  2747.0
index  1537 this word count in spam + 1 60.0 spam total:  1257.0
index  1537 likelihood_probs for index i 0.005824535857298871 0.0477326968973747
index  1538  this word count in non-spam + 1 34.0 non-spam total:  2747.0
index  1538 this word count in spam + 1 70.0 spam total:  1257.0
index  1538 likelihood_probs for index i 0.012377138696760102 0.05568814638027048
index  1539  this word count in non-spam + 1 14.0 non-spam total:  2747.0
index  1539 this word count in spam + 1 26.0 spam total:  1257.0
index  1539 likelihood_probs for index i 0.005096468875136513 0.020684168655529037
index  1540  this word count in no

index  1824 this word count in spam + 1 29.0 spam total:  1257.0
index  1824 likelihood_probs for index i 0.024026210411357846 0.023070803500397773
index  1825  this word count in non-spam + 1 23.0 non-spam total:  2747.0
index  1825 this word count in spam + 1 7.0 spam total:  1257.0
index  1825 likelihood_probs for index i 0.008372770294867127 0.005568814638027049
index  1826  this word count in non-spam + 1 30.0 non-spam total:  2747.0
index  1826 this word count in spam + 1 5.0 spam total:  1257.0
index  1826 likelihood_probs for index i 0.010921004732435385 0.003977724741447892
index  1827  this word count in non-spam + 1 53.0 non-spam total:  2747.0
index  1827 this word count in spam + 1 3.0 spam total:  1257.0
index  1827 likelihood_probs for index i 0.019293775027302514 0.002386634844868735
index  1828  this word count in non-spam + 1 59.0 non-spam total:  2747.0
index  1828 this word count in spam + 1 22.0 spam total:  1257.0
index  1828 likelihood_probs for index i 0.0214779

index  2094 this word count in spam + 1 21.0 spam total:  1257.0
index  2094 likelihood_probs for index i 0.011285038223516564 0.016706443914081145
index  2095  this word count in non-spam + 1 30.0 non-spam total:  2747.0
index  2095 this word count in spam + 1 3.0 spam total:  1257.0
index  2095 likelihood_probs for index i 0.010921004732435385 0.002386634844868735
index  2096  this word count in non-spam + 1 39.0 non-spam total:  2747.0
index  2096 this word count in spam + 1 32.0 spam total:  1257.0
index  2096 likelihood_probs for index i 0.014197306152166 0.02545743834526651
index  2097  this word count in non-spam + 1 44.0 non-spam total:  2747.0
index  2097 this word count in spam + 1 13.0 spam total:  1257.0
index  2097 likelihood_probs for index i 0.016017473607571896 0.010342084327764518
index  2098  this word count in non-spam + 1 68.0 non-spam total:  2747.0
index  2098 this word count in spam + 1 30.0 spam total:  1257.0
index  2098 likelihood_probs for index i 0.024754277

index  2359  this word count in non-spam + 1 29.0 non-spam total:  2747.0
index  2359 this word count in spam + 1 11.0 spam total:  1257.0
index  2359 likelihood_probs for index i 0.010556971241354204 0.008750994431185362
index  2360  this word count in non-spam + 1 25.0 non-spam total:  2747.0
index  2360 this word count in spam + 1 17.0 spam total:  1257.0
index  2360 likelihood_probs for index i 0.009100837277029487 0.013524264120922832
index  2361  this word count in non-spam + 1 19.0 non-spam total:  2747.0
index  2361 this word count in spam + 1 28.0 spam total:  1257.0
index  2361 likelihood_probs for index i 0.00691663633054241 0.022275258552108195
index  2362  this word count in non-spam + 1 47.0 non-spam total:  2747.0
index  2362 this word count in spam + 1 7.0 spam total:  1257.0
index  2362 likelihood_probs for index i 0.017109574080815437 0.005568814638027049
index  2363  this word count in non-spam + 1 13.0 non-spam total:  2747.0
index  2363 this word count in spam + 1 

In [ ]:
# plot the training and validation error rate vs. the thresholds
# choose the threshold with the minimal validation error rate and report the corresponding test error rate

# TODO
plt.subplot(2, 1, 1)
plt.plot(thresholds,train_error[::-1])
plt.xlabel("threshold")
plt.ylabel("train_error")
plt.title("train_error vs threshold")
plt.subplot(2, 1, 2)
plt.plot(thresholds,val_error[::-1],label="val error vs. threshold")
plt.xlabel("threshold")
plt.ylabel("val_error")
plt.title("val_error vs threshold")
plt.legend()
plt.show()

opt = thresholds[val_error.index(min(val_error))]

# print('Best performance at validated threshold {} with test error rate {}.'.format(opt, test_error[opt]))